# Task 04.05.20 Give Me Some Credit. Custom classes

In [37]:
#Выбросы (взгляд из Excel)
#NumberOfTime30-59DaysPastDueNotWorse, NumberOfTimes90DaysLate, NumberOfTime60-89DaysPastDueNotWorse = 96 или 98
#Очень большой DebtRatio, как правило, при MonthlyIncome = NA или 0. Импутация MonthlyIncome ситуацию не исправит, 
#т.к. не приведёт к пересчету DebtRatio.
#И пр.


In [38]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
os.getcwd()

'D:\\ML\\Gewissta\\My course\\GiveMeSomeCredit'

In [40]:
os.chdir('D:\\ML\\Gewissta\\My course\\GiveMeSomeCredit')

In [41]:
data = pd.read_csv('cs-training.csv', sep = ',')

In [42]:
data.tail()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
149995,149996,0,0.040674,74,0,0.225131,2100.0,4,0,1,0,0.0
149996,149997,0,0.299745,44,0,0.716562,5584.0,4,0,1,0,2.0
149997,149998,0,0.246044,58,0,3870.000000,NaN,18,0,1,0,0.0
149998,149999,0,0.000000,30,0,0.000000,5716.0,4,0,0,0,0.0
149999,150000,0,0.850283,64,0,0.249908,8158.0,8,0,2,0,0.0


In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 12 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Unnamed: 0                            150000 non-null  int64  
 1   SeriousDlqin2yrs                      150000 non-null  int64  
 2   RevolvingUtilizationOfUnsecuredLines  150000 non-null  float64
 3   age                                   150000 non-null  int64  
 4   NumberOfTime30-59DaysPastDueNotWorse  150000 non-null  int64  
 5   DebtRatio                             150000 non-null  float64
 6   MonthlyIncome                         120269 non-null  float64
 7   NumberOfOpenCreditLinesAndLoans       150000 non-null  int64  
 8   NumberOfTimes90DaysLate               150000 non-null  int64  
 9   NumberRealEstateLoansOrLines          150000 non-null  int64  
 10  NumberOfTime60-89DaysPastDueNotWorse  150000 non-null  int64  
 11  

In [44]:
data.isnull().sum()

Unnamed: 0                                  0
SeriousDlqin2yrs                            0
RevolvingUtilizationOfUnsecuredLines        0
age                                         0
NumberOfTime30-59DaysPastDueNotWorse        0
DebtRatio                                   0
MonthlyIncome                           29731
NumberOfOpenCreditLinesAndLoans             0
NumberOfTimes90DaysLate                     0
NumberRealEstateLoansOrLines                0
NumberOfTime60-89DaysPastDueNotWorse        0
NumberOfDependents                       3924
dtype: int64

In [45]:
data.duplicated().sum()

0

In [46]:
num_columns_1 = ['MonthlyIncome']
num_columns_2 = ['NumberOfDependents']
num_columns_age = ['age']
num_columns_3 = ['RevolvingUtilizationOfUnsecuredLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 
                'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 
                'NumberOfTime60-89DaysPastDueNotWorse']

In [47]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('SeriousDlqin2yrs', axis = 1), 
                                                    data['SeriousDlqin2yrs'], 
                                                    test_size = 0.3, 
                                                    stratify = data['SeriousDlqin2yrs'],
                                                    random_state = 42)

In [48]:
# создаем собственный класс, выполняющий скорректированное 
# логарифмическое преобразование
class CorrLog(BaseEstimator, TransformerMixin):
    """
    Параметры:
    lower: задает нижнее пороговое значение
    k: задает корректировку асимметрии
    copy: возвращает копию
    """
    def __init__(self, lower=0.001, k=0.2, copy=True):
        # все параметры для инициализации публичных атрибутов 
        # должны быть заданы в методе __init__
        
        # публичные атрибуты
        self.lower = lower
        self.k = k
        self.copy = copy
                
    def fit(self, X, y=None):
        # fit должен принимать в качестве аргументов только X и y
        # даже если ваша модель является моделью машинного обучения 
        # без учителя, вы должны принять аргумент y, это требуется 
        # для совместимости с конвейерами!
        
        # обучение модели осуществляется прямо здесь
        
        # создаем пустой словарь, в котором ключами
        # будут имена/целые числа, а значениями - средние
        self._encoder_dict = {}
        
        # функция isinstance() проверяет, является ли
        # объект датафреймом pandas или нет
        if isinstance(X, pd.DataFrame):
            for col in X.columns:
                # вычисляем среднее и записываем в словарь
                self._encoder_dict[col] = np.mean(X[col])
        else:
            for col in range(X.shape[1]):
                # вычисляем среднее и записываем в словарь
                self._encoder_dict[col] = np.mean(X[:, col])
            
        # fit возвращает self
        return self
    
    def transform(self, X):
        # transform принимает в качестве аргумента только X
        
        # выполняем копирование массива во избежание предупреждения SettingWithCopyWarning
        # "A value is trying to be set on a copy of a slice from a DataFrame (Происходит 
        # попытка изменить значение в копии среза данных датафрейма)"
        if self.copy:
            X = X.copy()
        
        # применяем преобразование к X
        if isinstance(X, pd.DataFrame):
            for col in X.columns:
                X[col] = np.log(np.clip(X[col], self.lower, None) / self._encoder_dict[col] + self.k)
        else:    
            for col in range(X.shape[1]):
                X[:, col] = np.log(np.clip(X[:, col], self.lower, None) / self._encoder_dict[col] + self.k)
                           
        return X

In [49]:
# создаем собственный класс, заменяющий отрицательные
# и нулевые значения на небольшие положительные
class Replacer(BaseEstimator, TransformerMixin):
    """
    Параметры:
        repl_value: значение для замены.
    """
    def __init__(self, repl_value=0.1):
        self.repl_value = repl_value
        
    def fit(self, X, y=None):
        # fit здесь бездельничает
        return self
    
    def transform(self, X):
        # transform выполняет всю работу: применяет преобразование 
        # с помощью заданного значения параметра repl_value
        X_replaced = np.where(X <= 0, self.repl_value, X)
        return X_replaced

In [50]:
# создаем собственный класс, выполняющий биннинг
class CustomBinning(BaseEstimator, TransformerMixin):
    """
    Параметры:
        bins: список бинов.
    """
    def __init__(self, bins=[-np.inf,  2, 4, np.inf]):
        self.bins = bins
    
    def fit(self, X, y=None):
        # fit опять бездельничает
        return self
    
    def transform(self, X):
        bins = np.array(self.bins)
         # transform выполняет всю работу: применяет преобразование 
        # с помощью заданного значения параметра bins
        X_bin = np.digitize(X, bins)
        return X_bin

In [51]:
num_pipe_1 = Pipeline([('imp', SimpleImputer(strategy = 'mean')),
                     ('corrlog', CorrLog(k=0.001)),
                     ('scaler', StandardScaler())])

In [52]:
num_pipe_2 = Pipeline([('imp', SimpleImputer(strategy = 'most_frequent')), 
                     ('binn', CustomBinning()),
                     ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

In [53]:
num_pipe_age = Pipeline([('imp', SimpleImputer(strategy = 'mean')), 
                       ('binn', CustomBinning()),
                       ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

In [54]:
num_pipe_3 = Pipeline([('imp', SimpleImputer(strategy='mean')),
                     ('repl', Replacer()),
                     ('box_cox', PowerTransformer(method='box-cox', standardize=True))])

In [55]:
transformers = [('num_1', num_pipe_1, num_columns_1), 
                ('num_2', num_pipe_2, num_columns_2), 
                ('num_age', num_pipe_age, num_columns_age), 
                ('num_3', num_pipe_3, num_columns_3)]

In [56]:
transformer = ColumnTransformer(transformers = transformers)

In [57]:
ml_pipe = Pipeline([('tf', transformer),
                    ('logreg', LogisticRegression(solver = 'lbfgs', max_iter = 200))])

In [63]:
param_grid = {
    #'logreg__C': [0.1, 0.2, 0.3],
    #'logreg__solver': ['lbfgs', 'liblinear'],
    #'tf__num_1__corrlog__k': [0.1, 0.5, 0.6, 0.7, 0.8],
    'tf__num_2__binn__bins': [[-np.inf, 2, 6, 10, np.inf], 
                             [-np.inf, 3, 5, np.inf]],
    'tf__num_age__binn__bins': [[-np.inf, 20, 40, 60, 80, np.inf], 
                             [-np.inf, 20, 50, 80, np.inf]],
    'tf__num_3__repl__repl_value': [0.005, 0.01, 0.2]
}

In [64]:
gs = GridSearchCV(ml_pipe, 
                  param_grid, 
                  scoring='roc_auc', 
                  cv=5, 
                  return_train_score=False)

# выполняем решетчатый поиск
gs.fit(X_train, y_train)

# смотрим наилучшие значения гиперпараметров
print('Наилучшие значения гиперпараметров: {}'.format(gs.best_params_))
# смотрим наилучшее значение AUC
print('Наилучшее значение AUC: {:.3f}'.format(gs.best_score_))
# смотрим значение AUC на тестовой выборке
print('AUC на тестовой выборке: {:.3f}'.format(
    roc_auc_score(y_test, gs.predict_proba(X_test)[:, 1])))

Наилучшие значения гиперпараметров: {'tf__num_2__binn__bins': [-inf, 3, 5, inf], 'tf__num_3__repl__repl_value': 0.2, 'tf__num_age__binn__bins': [-inf, 20, 40, 60, 80, inf]}
Наилучшее значение AUC: 0.848
AUC на тестовой выборке: 0.852
